<a href="https://colab.research.google.com/github/ruslanmv/Finetune-DeepSeek-R1-for-Reasoning/blob/master/Finetune-Granite3.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetune ibm granite 3.1 for Reasoning
by [ruslanmv.com](https://ruslanmv.com/)
![](https://images.pexels.com/photos/1181271/pexels-photo-1181271.jpeg)
## A Step-by-Step Guide


# Step 1: Installation

In [1]:
# %%capture
# Skip restarting message in Colab
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install unsloth vllm
!pip install --upgrade pillow
# If you are running this notebook on local, you need to install `diffusers` too
# !pip install diffusers
# Temporarily install a specific TRL nightly version
!pip install git+[https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b](https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b)

/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `pip install git+[https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b](https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b)'


# Step 2: Prepare for GRPO with Unsloth Patch

In [1]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-11 22:30:03 __init__.py:190] Automatically detected platform cuda.


# Step 3: Load ibm-granite/granite-3.1-8b-instruct Model

In [2]:
from unsloth import is_bfloat16_supported
import torch

max_seq_length = 512 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

In [4]:
model, tokenizer = FastLanguageModel.from_pretrained(
   #model_name="ibm-granite/granite-3.1-8b-instruct",
    model_name="ibm-granite/granite-3.1-2b-instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # Reduce if out of memory
)

==((====))==  Unsloth 2025.2.5: Fast Granite patching. Transformers: 4.48.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading ibm-granite/granite-3.1-2b-instruct with actual GPU utilization = 57.07%
Unsloth: Your GPU has CUDA compute capability 8.0 with VRAM = 39.56 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 512. Num Sequences = 288.
Unsloth: vLLM's KV Cache can use up to 17.75 GB. Also swap space = 6 GB.
INFO 02-11 22:30:42 config.py:542] This model supports multiple tasks: {'generate', 'reward', 'classify', 'score', 'embed'}. Defaulting to 'generate'.
INFO 02-11 22:30:42 llm_engine.py:234] Initializing

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 02-11 22:30:45 model_runner.py:1115] Loading model weights took 4.7360 GB
INFO 02-11 22:30:45 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 02-11 22:30:48 worker.py:267] Memory profiling takes 2.53 seconds
INFO 02-11 22:30:48 worker.py:267] the current vLLM instance can use total_gpu_memory (39.56GiB) x gpu_memory_utilization (0.57) = 22.57GiB
INFO 02-11 22:30:48 worker.py:267] model weights take 4.74GiB; non_torch_memory takes 0.02GiB; PyTorch activation peak memory takes 0.51GiB; the rest of the memory reserved for KV Cache is 17.31GiB.
INFO 02-11 22:30:49 executor_base.py:110] # CUDA blocks: 14177, # CPU blocks: 4915
INFO 02-11 22:30:49 executor_base.py:115] Maximum concurrency for 512 tokens per request: 443.03x
INFO 02-11 22:30:54 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error 

Capturing CUDA graph shapes: 100%|██████████| 39/39 [00:47<00:00,  1.23s/it]

INFO 02-11 22:31:42 model_runner.py:1562] Graph capturing finished in 48 secs, took 0.55 GiB
INFO 02-11 22:31:42 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 56.60 seconds


ibm-granite/granite-3.1-2b-instruct does not have a padding token! Will use pad_token = <|end_of_text|>.


In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

Unsloth 2025.2.5 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


# Step 4: Data Preparation

In [6]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?<\/reasoning>\s*<answer>.*?<\/answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

# Step 5: Fine-tuning with GRPO Trainer

In [7]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 6, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

torch.distributed process group is initialized, but parallel_mode != ParallelMode.DISTRIBUTED. In order to use Torch DDP, launch your script with `python -m torch.distributed.launch


In [8]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)

trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 1
\        /    Total batch size = 1 | Total steps = 250
 "-____-"     Number of trainable parameters = 56,360,960


-------------------- Question:
Ahmed and Emily are having a contest to see who can get the best grade in the class. There have been 9 assignments and Ahmed has a 91 in the class. Emily has a 92. The final assignment is worth the same amount as all the other assignments. Emily got a 90 on the final assignment. What is the minimum grade Ahmed needs to get to beat Emily if all grades are whole numbers? 
Answer:
100 
Response:
<reasoning>
To find the minimum grade Ahmed needs to beat Emily, we first need to determine Emily's total points after all 9 assignments, considering her final grade. Since her final grade is 90 and the class average is 91, it means she scored 1 point lower than average on each of the 9 assignments. Thus, Emily's total points from these assignments are:

9 (assignments) * (91 - 1) = 819 points

Now, including her final grade, Emily's total points are:

819 + 90 = 909 points

Ahmed needs to beat Emily by at least 1 point, so the minimum grade he needs on the final ass

Step,Training Loss,reward,reward_std,completion_length,kl
1,0.005000,0.085500,0.231796,198.000000,0.126040
2,0.005500,0.070333,0.085451,195.166672,0.137374
3,0.003000,0.407500,1.048033,178.166672,0.075547
4,0.004000,0.125000,0.000000,200.000000,0.101067
5,0.001300,0.104167,0.051031,200.000000,0.033107
6,0.001900,0.079167,0.112268,200.000000,0.047467
7,0.003500,0.032333,0.089866,186.000000,0.086650
8,0.002100,0.445167,0.996250,190.333344,0.052074
9,0.002900,-0.011833,0.121628,179.000000,0.073284
10,0.006100,0.599667,1.058371,117.166672,0.151964


Streaming output truncated to the last 5000 lines.
Extracted:
John uses the bathroom 3 times during a 2.5-hour movie.
-------------------- Question:
There is a 5,200 sq. ft. house and a 7,300 sq. ft. house next to each other. The smaller house is being expanded. If the new total square footage of both houses is 16,000 sq. ft., how much is the smaller house being expanded by, in sq. ft.? 
Answer:
3500 
Response:
<reasoning>
First, let's find out the current total square footage of both houses before the expansion. The smaller house has 5,200 sq. ft. and the larger one has 7,300 sq. ft., so together they have 5,200 + 7,300 = 12,500 sq. ft.

Next, we know that after the expansion, both houses will have a total of 16,000 sq. ft.

To find out how much the smaller house is being expanded by, subtract its current square footage from the combined future square footage: 16,000 - 12,500 = 3,500 sq. ft.

So, the smaller house is being expanded by 3,500 sq. ft.
</reasoning>

<answer>3 
Extracted:


TrainOutput(global_step=250, training_loss=0.042628783367108555, metrics={'train_runtime': 781.7706, 'train_samples_per_second': 0.32, 'train_steps_per_second': 0.32, 'total_flos': 0.0, 'train_loss': 0.042628783367108555})

# Step 6: Inference

First, let's first try the model without any GRPO trained:

In [10]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

print(output)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.15s/it, est. speed input: 54.10 toks/s, output: 102.09 toks/s]

As a text-based AI, I don't have the ability to perform calculations directly. However, I can tell you that the value of pi (π) is approximately 3.14159, which has been calculated and estimated through various mathematical methods and infinite series. The first known calculation of pi was performed by the ancient Greek mathematician Archimedes, who used a polygon inscribed within and outside a circle to approximate the value. His method involved calculating the perimeters of these polygons and taking the limit as the number of sides increased.


And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [9]:
model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [11]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

print(output)

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.83s/it, est. speed input: 9.53 toks/s, output: 96.71 toks/s]

<reasoning>

Pi (π) is a mathematical constant representing the ratio of a circle's circumference to its diameter. It cannot be precisely calculated to an infinite number of decimal places due to its transcendental nature. However, we can use mathematical formulas and infinite series to approximate its value.

One popular method is the Gregory-Leibniz series:

π/4 = 1 - 1/3 + 1/5 - 1/7 + 1/9 - 1/11 + ...

By summing up an extremely large number of terms, we can get increasingly accurate approximations of π.

Another method is the Chudnovsky algorithm:

π = 12 * sum from n=0 to ∞ of ((-1)^n * (6n)! * (545140134n + 13591409)) / ((n!^3 * 3^12 * 640320^3))

Using computational tools, the most accurate approximation of π (up to 100 trillion decimal places) is:

π ≈ 3.1415926535897932384626433832795028841971693993751058209749445923078164062862089986280348253421170679

</reasoning>

<answer>
3.1415926535897932384626433832795028841971693993751058209749445923078164062862089986280348253


# Step 7: Saving to float16 for VLLM

In [ ]:
# Merge to 16bit
# if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
# if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
# if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
# if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
# if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
# if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

In [ ]:
from google.colab import userdata
HF_TOKEN=userdata.get('HF_TOKEN')

In [12]:
 model.save_pretrained_merged("model", tokenizer, save_method = "lora",)

Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... Done.


In [16]:
model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)

Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.1G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 45.21 out of 83.48 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 40/40 [00:00<00:00, 96.23it/s] 


Unsloth: Saving tokenizer... Done.
Done.


In [20]:
model.push_to_hub_merged(
    "ruslanmv/granite-3.1-2b-Reasoning",
    tokenizer,
    save_method= "merged_16bit",
    token=HF_TOKEN,
    commit_message="First commit",
    commit_description="Uploading model GRO"
)

Unsloth: You are pushing to hub, but you passed your HF username = ruslanmv.
We shall truncate ruslanmv/granite-3.1-2b-Reasoning to granite-3.1-2b-Reasoning


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 45.06 out of 83.48 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 40/40 [00:00<00:00, 151.08it/s]


Unsloth: Saving tokenizer... Done.


README.md:   0%|          | 0.00/595 [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/101M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/ruslanmv/granite-3.1-2b-Reasoning


In [18]:
model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)

Help on method unsloth_push_to_hub_merged in module unsloth.save:

unsloth_push_to_hub_merged(repo_id: str, tokenizer=None, save_method: str = 'merged_16bit', use_temp_dir: Optional[bool] = None, commit_message: Optional[str] = 'Trained with Unsloth', private: Optional[bool] = None, token: Union[bool, str, NoneType] = None, max_shard_size: Union[int, str, NoneType] = '5GB', create_pr: bool = False, safe_serialization: bool = True, revision: str = None, commit_description: str = 'Upload model trained with Unsloth 2x faster', tags: Optional[List[str]] = None, temporary_location: str = '_unsloth_temporary_saved_buffers', maximum_memory_usage: float = 0.75) method of peft.peft_model.PeftModelForCausalLM instance
    Same as .push_to_hub(...) except 4bit weights are auto
    converted to float16 with as few overhead as possible.
    
    Choose for `save_method` to be either:
    1. `16bit`: Merge LoRA into float16 weights. Useful for GGUF / llama.cpp.
    2.  `4bit`: Merge LoRA into int4 w

In [24]:
model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit_forced",)


Unsloth: Merging 4bit and LoRA weights to 4bit...
This might take 5 minutes...
Done.
Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 10 minutes for Llama-7b... Done.


In [25]:
model.push_to_hub_merged(
    "ruslanmv/granite-3.1-2b-Reasoning-4bit",
    tokenizer,
    save_method="merged_4bit_forced",
    token=HF_TOKEN,
    commit_message="First commit"
)

Unsloth: Merging 4bit and LoRA weights to 4bit...
This might take 5 minutes...
Done.
Unsloth: Saving 4bit Bitsandbytes model. Please wait...


README.md:   0%|          | 0.00/595 [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/101M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/602 [00:00<?, ?B/s]

Saved merged_4bit model to https://huggingface.co/ruslanmv/granite-3.1-2b-Reasoning-4bit


In [ ]:
model.push_to_hub_merged("ruslanmv/granite-3.1-2b-Reasoning", tokenizer, save_method = "lora", token = HF_TOKEN)

In [15]:
model.push_to_hub_merged(
    "ruslanmv/granite-3.1-2b-Reasoning-LORA",
    tokenizer,
    save_method="lora",
    token=HF_TOKEN,
    commit_message="First commit"
)


Unsloth: Saving LoRA adapters. Please wait...


README.md:   0%|          | 0.00/595 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/226M [00:00<?, ?B/s]

Saved lora model to https://huggingface.co/ruslanmv/granite-3.1-2b-Reasoning-LORA


# GGUF / llama.cpp Conversion

In [ ]:
# Save to 8bit Q8_0
# if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to [https://huggingface.co/settings/tokens](https://huggingface.co/settings/tokens) for a token!
# And change hf to your username!
# if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
# if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
# if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
# if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
# if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
# if False:
#     model.push_to_hub_gguf(
#         "hf/model", # Change hf to your username!
#         tokenizer,
#         quantization_method = ["q4_k_m", "q8_0", "q5_k_m"],
#         token = "",
#     )

In [21]:
model.push_to_hub_gguf(
         "ruslanmv/granite-3.1-2b-Reasoning-GGUF", # Change hf to your username!
         tokenizer,
         quantization_method = ["q4_k_m", "q8_0", "q5_k_m"],
         token = HF_TOKEN,
         commit_message="First commit",
     )

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 50.79 out of 83.48 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 40/40 [00:00<00:00, 150.06it/s]


Unsloth: Saving tokenizer... Done.
Done.


Unsloth: Converting granite model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m', 'q8_0', 'q5_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at ruslanmv/granite-3.1-2b-Reasoning-GGUF into bf16 GGUF format.
The output location will be /content/ruslanmv/granite-3.1-2b-Reasoning-GGUF/unsloth.BF16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: granite-3.1-2b-Reasoning-GGUF
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'mod

...

Saved GGUF to https://huggingface.co/ruslanmv/granite-3.1-2b-Reasoning-GGUF
Unsloth: Uploading GGUF to Huggingface Hub...


  0%|          | 0/1 [00:00<?, ?it/s]

unsloth.Q5_K_M.gguf:   0%|          | 0.00/1.80G [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Saved GGUF to https://huggingface.co/ruslanmv/granite-3.1-2b-Reasoning-GGUF
